In [1]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

from matplotlib.colors import LinearSegmentedColormap

from scipy.stats import binom

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [2]:
omdf = pd.read_excel(rootdir_data+'Omikron.xlsx')
omdf['Dato'] = pd.to_datetime(omdf['Dato'])
omdf['Ratio'] = omdf['AntalOmikron']/omdf['AntalTest']
omdf['Perc'] = 100 * omdf['AntalOmikron']/omdf['AntalTest']


omdf = omdf.iloc[:-2]

# Stan tests

In [3]:
import pystan

In [4]:
stanModel = """
data {
int<lower=0> N;
vector[N] x;
vector[N] y;
}
parameters {
real alpha;
real beta;
real<lower=0> sigma;
}
model {
y ~ normal(alpha + beta * x, sigma);
}
"""

In [ ]:
# posterior = pystan.build(stanModel,data=stan_data)
sm = pystan.StanModel(model_code=stanModel)

In [75]:
a = 0.4
b = 2.5
s = 10

# x = np.arange(0,20)
numVals = 30
x = np.linspace(0,20,numVals)
curNoise = np.random.normal(0,s,x.shape)
yTrue = a + b * x 
yNoise = yTrue + curNoise

plt.figure(figsize=(10,5))
plt.plot(x,yTrue)
plt.plot(x,yNoise,'*')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
stan_data = {   "N" : numVals,
                "x" : x,
                "y" : yNoise}

In [77]:
fit = sm.sampling(data=stan_data)

In [78]:
fit2 = sm.sampling(data=stan_data, iter=10000, chains=4)

In [79]:
la = fit2.extract(permuted=True)  # return a dictionary of arrays
# mu = la['mu']

## return an array of three dimensions: iterations, chains, parameters
a = fit2.extract(permuted=False)

In [80]:
# la['alpha']

# plt.figure()

# plt.plot(la['alpha'])

numChainToSample = 5000
curLen = len(la['alpha'])
# for k in range(numChainToSample):

posVals = np.arange(curLen)
sampleIndices= np.random.choice(posVals,size=(numChainToSample,))

# plt.figure()
# for i in sampleIndices:
#     cA = la['alpha'][i]
#     cB = la['beta'][i]

#     cY = cA + cB * x
#     plt.plot(x,cY,'k',alpha=0.1)

allYs = []
for i in sampleIndices:
    cA = la['alpha'][i]
    cB = la['beta'][i]

    cY = cA + cB * x

    allYs.append(cY)
allYs = np.array(allYs)

# np.mean(allYs,axis=0).shape
# np.mean(allYs,axis=0)

# print(np.quantile(allYs,.50,axis=0))
# print(np.median(allYs,axis=0))

plt.figure()
# plt.plot(x,np.mean(allYs,axis=0),'k')
# plt.plot(x,np.median(allYs,axis=0),'k')

# plt.plot(x,np.quantile(allYs,0.9,axis=0),color='grey')
# plt.plot(x,np.quantile(allYs,1-0.9,axis=0),color='grey')

curP = 0.99
plt.fill_between(x,np.quantile(allYs,curP,axis=0),np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
curP = 0.95
plt.fill_between(x,np.quantile(allYs,curP,axis=0),np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
curP = 0.9
plt.fill_between(x,np.quantile(allYs,curP,axis=0),np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
curP = 0.75
plt.fill_between(x,np.quantile(allYs,curP,axis=0),np.quantile(allYs,1-curP,axis=0),color='k',edgecolor='none',alpha=0.2)
plt.plot(x,np.median(allYs,axis=0),'k')
plt.plot(x,yTrue,'b*:')
plt.plot(x,yNoise,'m.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …